# Part 1: Getting the dataset

### Download the SFPD dataset and read the csv in Python

In [1]:
import csv
import pandas as pd
from matplotlib import pyplot as plt
from collections import Counter
import operator
import numpy as np
%matplotlib inline

In [8]:
# Define the data path
data_path = 'SFPD_data.csv'

In [9]:
# Create a function that reads a csv to a dictionary
def parse_csv(datafile):
    with open(datafile, 'rb') as sd:
        r = csv.DictReader(sd)
        data = [line for line in r]
    return data

In [10]:
# Or load it into a Dataframe using pandas
raw_file = pd.read_csv(data_path)
df = pd.DataFrame(raw_file)
df.head(5)

IOError: File ..\week_3\SFPD_data.csv does not exist

# Part 2 : Working with data

In [ ]:
# But let's better work with dictionaries as we go forward
sfpd_dict = parse_csv(data_path)

In [ ]:
# See how the dictionary looks like
sfpd_dict[:1]

### The types of crime and their popularity over time. The first field we'll dig into is the column "Category".

* *Create a set() of all the categories of crime in the dataset. How many are there?*

In [ ]:
# Instead of set() we use unique of pandas
print 'The unique categories are: %d \n\nThey are the following: \n%s' % (len(df.Category.unique()), df.Category.unique())

* *Now count the number of occurrences of each category in the dataset. What is the most commonly occurring category of crime? What is the least frequently occurring?*

In [ ]:
# Can be done with pandas with reset_index to turn Series into a dataframe
crime_occurences = df['Category'].groupby(df['Category']).count().sort_values(ascending=1).reset_index(name='count')
crime_occurences

* *Create a histogram over crime occurrences.*

In [ ]:
# Assign x and y values
x = crime_occurences['Category']
y = crime_occurences['count']

# Give indeces to crimes
x_pos = np.arange(len(x))

# Assign the size of the plot before declaring the plot
plt.figure(figsize= (9,5))

# Give the x and y values to the barplot
plt.bar(x_pos, y,align='center', color='grey', alpha=.5)

# Match the indeces with the correct courses and assign them to the x-axis
plt.xticks(x_pos, x, rotation='vertical')

# Give labels and titles
plt.ylabel("Crime Count")
plt.title("Frequency of crime categories")

# Show the barplot
plt.show()

* *Now it's time to explore how the crime statistics change over time. To start off easily, let's count the number of crimes per year for the years 2003-2016 (the full years in the database). For that one, I used the date column in the CSV file and used year = int( date.split("/")[-1] ) to extract the year of each crime. What's the average number of crimes per year?*

In [ ]:
# Turn Date into a datetime type instead of string
#df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Create a function that gets the date
def getYear(s):
    return int(s.split('/')[-1])

In [ ]:
# Now create a new column named year and store the values of the above function
df['Year']= df['Date'].apply(lambda x: getYear(x))

In [ ]:
# List the number of crimes per year
crime_occurences_time = df.groupby('Year')['Year'].count().reset_index(name='count').sort_values(by='Year', ascending=1)
crime_occurences_time

* *Police chief Suneman is interested in the temporal development of only a subset of categories, the so-called focus crimes. Those categories are listed below (for convenient copy-paste action). Now create bar-charts displaying the year-by-year development of each of these categories across the years 2003-2016.*

In [ ]:
focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

In [ ]:
# Keep only the Categories of the list in a seperate dataframe
focus_df = df.filter(items=['Category', 'Year']) 
focus_df = focus_df[focus_df['Category'].isin(focuscrimes)]
focus_df.head()

In [ ]:
plt.figure(figsize = (12,12))
plt.suptitle('Frequency of crime categories over the years')
for index,crime in enumerate(focuscrimes):
    # Create a matrix of subplots with 7 rows and two columns
    plt.subplot(7,2,index+1)
     
    # Create a sub-df for each category
    temp_df = focus_df[focus_df['Category']==crime]
    
    # Drop column category from the new df to keep only year
    temp_df = temp_df.drop('Category', axis=1) # axis=1 means column
    
    # Drop 2017 as it could be misleading as only few values for the first two months exist
    temp_df = temp_df[temp_df.Year != 2017]
    
    # Count frequency of crime occurence
    occurence = temp_df.groupby('Year')['Year'].count().reset_index(name='count').sort_values(by='Year', ascending=1)
    
    # Assign x and y values
    x = occurence['Year']
    y = occurence['count']

    plt.bar(x,y,align='center',alpha=.5, color='grey',label=crime)
    plt.xticks(np.arange(2003,2017,2))
    plt.ylabel("Crime Counts")
    plt.legend(loc='upper right',frameon=False,handlelength=0, handletextpad=0)
    #plt.title("Linear regression dataset %d" %(i+1))
    
# Show the barplot
plt.show()

* *Comment on at least three interesting trends in your plot. Also, here's a fun fact: The drop in car thefts is due to new technology called 'engine immobilizer systems' - get the full story here.*

### The types of crime and their popularity across the 24 hours of the day

* *First, plot a simple histogram of the number of crimes per hour in the dataset. This piece of information is in the Time column of the CSV file, and I simply used hr = int( time.split(":")[0] ) to get this info out. There's a little strange peak at 12:00 - what do you think is going on there? (No need for a serious exploration, just some thoughts on what might be going on).*

In [ ]:
# Create a function that gets the time
def getHour(s):
    return int(s.split(':')[0])

In [ ]:
# Now create a new column named hour and store the values of the above function
df['Hour']= df['Time'].apply(lambda x: getHour(x))

In [ ]:
# List the number of crimes per year
crime_occurences_hour = df.groupby('Hour')['Hour'].count().reset_index(name='count').sort_values(by='Hour', ascending=1)

In [ ]:
# Assign x and y values
x = crime_occurences_hour['Hour']
y = crime_occurences_hour['count']

# Assign the size of the plot before declaring the plot
plt.figure(figsize= (9,5))

# Give the x and y values to the barplot
plt.bar(x, y, align = 'center', color='grey', alpha=.5)

# Give labels and titles
plt.ylabel("Crime Count")
plt.title("Frequency of crimes per hour")
plt.xticks(range(0,24,1))

# Show the barplot
plt.show()

* *Next create a plot of the count of each crime category across the 24 hours of the day. Again, comment on at least three trends in the data. *

In [ ]:
# Keep only the Categories of the list in a seperate dataframe
focus_df2 = df.filter(items=['Category', 'Hour']) 
focus_df2 = focus_df2[focus_df2['Category'].isin(focuscrimes)]
focus_df2.head()

In [ ]:
plt.figure(figsize = (12,12))
plt.suptitle('Frequency of crime categories among hours of the day')
for index,crime in enumerate(focuscrimes):
    # Create a matrix of subplots with 7 rows and two columns
    plt.subplot(7,2,index+1)
     
    # Create a sub-df for each category
    temp_df2 = focus_df2[focus_df2['Category']==crime]
    
    # Drop column category from the new df to keep only year
    temp_df2 = temp_df2.drop('Category', axis=1) # axis=1 means column
    
    # Drop 2017 as it could be misleading as only few values for the first two months exist
    temp_df2 = temp_df2[temp_df2.Hour != 2017]
    
    # Count frequency of crime occurence
    occurence = temp_df2.groupby('Hour')['Hour'].count().reset_index(name='count').sort_values(by='Hour', ascending=1)
    
    # Assign x and y values
    x = occurence['Hour']
    y = occurence['count']

    plt.bar(x,y,align='center',alpha=.5, color='grey',label=crime)
    plt.xticks(np.arange(0,24,5))
    plt.ylabel("Crime Counts")
    plt.legend(loc='upper right',frameon=False,handlelength=0, handletextpad=0)
    #plt.title("Linear regression dataset %d" %(i+1))
    
# Show the barplot
plt.show()

### The types of crime and how they take place across San Francisco's police districts.

* *So now we'll be combining information about PdDistrict and Category to explore differences between SF's >neighborhoods. First, simply list the names of SF's 10 police districts.*

In [ ]:
# Create a list of unique values of districts excluding nulls
districts = [i for i in df.PdDistrict.unique() if pd.notnull(i)]
print 'The %d police districts of SanFrancisco are: \n%s' % (len(districts),districts)

* *Which has the most crimes? Which has the most focus crimes?*

In [ ]:
# Keep only the Category and the District column
catdist_df = df.filter(items=['Category', 'PdDistrict']) 
catdist_df = catdist_df[catdist_df['PdDistrict'].isin(districts)]

# List the number of crimes per District
crime_occurences_district = catdist_df.groupby('PdDistrict')['PdDistrict'].count().reset_index(name='count').sort_values(by='count', ascending=0)

# Print the district with the most crimes
print 'The districts with the most crimes are:\n'
crime_occurences_district = crime_occurences_district.reset_index().drop('index', axis=1)
crime_occurences_district[:5]

In [ ]:
# Filter out only the focus crimes
catdistfocus_df = catdist_df[catdist_df['Category'].isin(focuscrimes)]

# List the number of crimes per District
crime_occurences_district_focus = catdistfocus_df.groupby('PdDistrict')['PdDistrict'].count().reset_index(name='count').sort_values(by='count', ascending=0)

# Print the district with the most focus crimes
print 'The districts with the most focus crimes are:\n'
crime_occurences_district_focus = crime_occurences_district_focus.reset_index().drop('index', axis=1)
crime_occurences_district_focus[:5]

* *Next, we want to generate a slightly more complicated graphic. I'm interested to know if there are certain crimes >that happen much more in certain neighborhoods than what's typical. Below I describe how to get that plot going:*

> First, we need to calculate the relative probabilities of seeing each type of crime in the dataset as a whole. > That's simply a normalized version of this plot. Let's call it P(crime)

In [ ]:
# Create one more column with the relative probability of seeing each type of crime
crime_occurences_district['P(crime)'] = crime_occurences_district['count']/crime_occurences_district['count'].sum()
crime_occurences_district[:5]

* *Next, we calculate that same probability distribution but for each PD district, let's call that P(crime|district).*